# Day1

In [29]:
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS
import pandas as pd
from sklearn.cluster import KMeans

In [3]:
%run src/file_utils.py
%run src/configuration.py

In [49]:
document_test = ['BMW-AnnualReport-2016.json']

In [24]:
vocab_documents = ['BMW-AnnualReport-2016.json', 'CarlZeissMeditec-AnnualReport-2016.json', 'BVB-AnnualReport-2016.json']

In [4]:
documents = ['BMW-AnnualReport-2015.json', 'BMW-AnnualReport-2016.json', 'BMW-AnnualReport-2017.json', 
 'CarlZeissMeditec-AnnualReport-2015.json', 'CarlZeissMeditec-AnnualReport-2016.json', 'CarlZeissMeditec-AnnualReport-2017.json',
 'BVB-AnnualReport-2015.json', 'BVB-AnnualReport-2016.json', 'BVB-AnnualReport-2017.json']

In [5]:
TYPE = 'type'
PARAGRAPH = 'paragraph'
CONTENT = 'content'

In [6]:
def readContentOfFile(file_name):
    content = ''
    try:
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    except:
        FileUtils.fix_json(file_name)
        with open(file_name) as f:
            data = json.load(f)
            for item in data:
                typeDoc = item[TYPE]
                if typeDoc == PARAGRAPH:
                    content += item[CONTENT]
    return content

In [40]:
common_vocabularly = set()
for document in documents:
    content_of_document = readContentOfFile(FILE_PATH + document)
    nlp = spacy.load("de")
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
    vocabularly = set()
    for word in filtered_words_withoutpunc:
        vocabularly.add(word.text.replace('\n', '').strip().lower())
    new_vocab = set()
    for u in vocabularly:
        if u != '':
            new_vocab.add(u)
    vectorizer = TfidfVectorizer(vocabulary=new_vocab)
    tfidf_matrix = vectorizer.fit_transform([content_of_document])
    feature_names = vectorizer.get_feature_names()
    corpus_index = [n for n in ['Values']]
    df = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
    df = df.sort_values(by=['Values'], ascending=False)
    common_vocabularly.update(df.head(1000).index.values.tolist())


In [42]:
vectorizer = TfidfVectorizer(vocabulary=common_vocabularly)

In [46]:
tfidf_matrix = vectorizer.fit_transform(
    [readContentOfFile(FILE_PATH + 'BVB-AnnualReport-2011.json'), 
     readContentOfFile(FILE_PATH + 'BVB-AnnualReport-2012.json'), 
     readContentOfFile(FILE_PATH + 'BMW-AnnualReport-2012.json'),
     readContentOfFile(FILE_PATH + 'BMW-AnnualReport-2011.json'),
     readContentOfFile(FILE_PATH + 'CarlZeissMeditec-AnnualReport-2013.json'),
     readContentOfFile(FILE_PATH + 'CarlZeissMeditec-AnnualReport-2012.json'),
     readContentOfFile(FILE_PATH + 'BVB-AnnualReport-2013.json'), 
     readContentOfFile(FILE_PATH + 'BMW-AnnualReport-2013.json'),])

In [47]:
km = KMeans(n_clusters=3, init='k-means++')
km.fit(tfidf_matrix)
km.labels_

array([1, 1, 2, 2, 0, 0, 1, 2], dtype=int32)

In [63]:
def perform_lemmatization(document):
    content_of_document = readContentOfFile(document)
    nlp = spacy.load("de")
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    lemmatized_content = " ".join(item for item in filtered_lemmas)
    return lemmatized_content

In [60]:
common_vocabularly_lem = set()
for document in documents:
    content_of_document = readContentOfFile(FILE_PATH + document)
    nlp = spacy.load("de")
    sentence = nlp(content_of_document)
    filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
    filtered_words_withoutdigits = [word for word in filtered_words if not word.is_digit]
    filtered_words_withoutpunc = [word for word in filtered_words_withoutdigits if word.pos_ != 'PUNCT']
    filtered_lemmas = [word.lemma_ for word in filtered_words_withoutpunc]
    vocabularly = set()
    for word in filtered_lemmas:
        vocabularly.add(word.replace('\n', '').strip().lower())
    new_vocab = set()
    for u in vocabularly:
        if u != '':
            new_vocab.add(u)

    lemmatized_content = " ".join(item for item in filtered_lemmas)
    vectorizer = TfidfVectorizer(vocabulary=new_vocab)
    tfidf_matrix = vectorizer.fit_transform([lemmatized_content])
    feature_names = vectorizer.get_feature_names()
    corpus_index = [n for n in ['Values']]
    df = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
    df = df.sort_values(by=['Values'], ascending=False)
    common_vocabularly_lem.update(df.head(1000).index.values.tolist())


In [61]:
vectorizer = TfidfVectorizer(vocabulary=common_vocabularly_lem)

In [64]:
tfidf_matrix = vectorizer.fit_transform(
    [perform_lemmatization(FILE_PATH + 'BVB-AnnualReport-2011.json'), 
     perform_lemmatization(FILE_PATH + 'BVB-AnnualReport-2012.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2012.json'),
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2011.json'),
     perform_lemmatization(FILE_PATH + 'CarlZeissMeditec-AnnualReport-2013.json'),
     perform_lemmatization(FILE_PATH + 'CarlZeissMeditec-AnnualReport-2012.json'),
     perform_lemmatization(FILE_PATH + 'BVB-AnnualReport-2013.json'), 
     perform_lemmatization(FILE_PATH + 'BMW-AnnualReport-2013.json'),])

In [66]:
km = KMeans(n_clusters=3, init='k-means++')
km.fit(tfidf_matrix)
km.labels_

array([0, 0, 1, 1, 2, 2, 0, 1], dtype=int32)

In [53]:
df.head(10)

,Values
bmw,0.655367
group,0.372566
million,0.252614
unternehmen,0.123924
segment,0.108831
risiko,0.108831
mitglied,0.094532
höhe,0.093737
fahrzeug,0.087382
vorstehen,0.086588


In [197]:
nlp = spacy.load("de")
sentence = nlp(content_of_all_documents)

In [198]:
filtered_words = [word for word in sentence if word.lower_ not in STOP_WORDS]
filtered_words_withoutpunc = [word for word in filtered_words if word.pos_ != 'PUNCT']

In [199]:
vocabularly = []
for word in filtered_words_withoutpunc:
    vocabularly.append(word.text.replace('\n', '').lower())

In [200]:
new_vocab = set()
for u in vocabularly:
    if u != '':
        new_vocab.add(u)

In [202]:
vectorizer = TfidfVectorizer(vocabulary=new_vocab)

In [232]:
tfidf_matrix = vectorizer.fit_transform([readContentOfFile(FILE_PATH + 'BVB-AnnualReport-2016.json'), readContentOfFile(FILE_PATH + 'BMW-AnnualReport-2016.json')])

In [233]:
print (len(new_vocab))

14420


In [234]:
feature_names = vectorizer.get_feature_names()
corpus_index = [n for n in ['A', 'B']]
import pandas as pd
df = pd.DataFrame(tfidf_matrix.T.todense(), index=feature_names, columns=corpus_index)
#df['1'].argmax()
# print(df.)

In [235]:
print (df['A'].argmax())

dortmund


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


In [238]:
print (df.sort_values(by=['B'], ascending=False))

                               A         B
bmw                     0.000000  0.778350
group                   0.000871  0.314828
2016                    0.160191  0.173860
2015                    0.147132  0.169833
automobile              0.000000  0.083967
unternehmen             0.040048  0.079211
höhe                    0.079225  0.079211
risiken                 0.060072  0.078539
vorstand                0.001741  0.073169
vorstands               0.000871  0.071826
aufsichtsrat            0.053107  0.066456
segment                 0.002612  0.062429
vorjahr                 0.084449  0.061086
geschäftsjahr           0.075743  0.061086
aufsichtsrats           0.010447  0.053702
beziehungsweise         0.000000  0.051890
entwicklung             0.017412  0.051017
fahrzeuge               0.000000  0.050947
mini                    0.000000  0.050947
wesentlichen            0.047883  0.050346
compliance              0.001741  0.049674
2017                    0.023506  0.047661
rahmen     